In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [2]:
dataset = pd.read_csv("FE_Salida.csv")
dataset.head()

,Unnamed: 0,bmi,Age,asa_status,baseline_cancer,baseline_charlson,baseline_cvd,baseline_dementia,baseline_diabetes,baseline_digestive,...,complication_rsi,dow,gender,hour,month,moonphase,mort30,mortality_rsi,race,complication
0,6603,2.037889,43.505845,1,0,0,1,0,0.0,0,...,-2.291575,0,1,0.612177,8,0,0,-2.307139,1,0
1,14615,2.162597,13.695847,0,0,0,0,0,0.0,0,...,-1.550571,2,0,0.612373,1,1,0,-0.870558,1,1
2,10064,2.153247,27.934089,0,0,0,1,0,0.0,0,...,-1.489581,3,0,0.609533,11,1,0,-0.574947,0,0
3,13156,2.077001,30.176871,1,0,0,0,0,0.0,0,...,0.000000,0,0,0.607329,8,1,0,0.000000,1,1
4,6800,2.037889,43.505845,1,0,0,1,0,0.0,0,...,-2.291575,0,1,0.612177,8,0,0,-2.307139,1,0


## Balanceo de data

In [3]:
dataset['complication'].value_counts()

0    7684
1    2560
Name: complication, dtype: int64

In [4]:
nYes = len(dataset[dataset['complication'] == 1])
yes = dataset[dataset['complication'] == 1]
no = dataset[dataset['complication'] == 0]
no = no.sample(2*nYes, random_state=2022)
dataset = no.append(yes)
dataset = dataset.sample(frac=1, random_state=2022)
dataset
dataset['complication'].value_counts()

0    5120
1    2560
Name: complication, dtype: int64

## Entrenamiento del modelo

In [5]:
X = dataset[['bmi', 'Age', 'baseline_cancer', 'baseline_charlson', 'baseline_osteoart', 'ccsComplicationRate', 'ccsMort30Rate', 'complication_rsi', 'mortality_rsi']]
y = dataset['complication']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=2022)

In [7]:
##parametros
gridLogReg = dict(solver=['lbfgs'],penalty=['l2'],C=[5, 2, 1.0, 0.1])
gridNB={
    'var_smoothing': np.logspace(0,-9, num=10)
}
gridDTreeCla= {'criterion':['gini'],'max_depth':[4,5,6,7,8,9,10]}
girdKNC=dict(leaf_size=list(range(1,10)), n_neighbors=list(range(1,6)), p=[1,2])
gridLDA=dict(solver=['svd'])
gridQDA = dict(reg_param=(0.00001, 0.0001, 0.001,0.01, 0.1), store_covariance=(True, False)) 

##Entrenamiento de modelos
svm = GridSearchCV(SVC(), {'C': [0.1, 1, 2, 5], 'kernel': ['linear']}, cv=2).fit(X_train, y_train)
randomForest = GridSearchCV(RandomForestClassifier(), {'n_estimators': [20], 'max_depth': [10]}, cv=2).fit(X_train, y_train)
logReg=GridSearchCV(LogisticRegression(), param_grid=gridLogReg, cv=2).fit(X_train, y_train)
gNB=GridSearchCV(GaussianNB(), param_grid=gridNB, cv=2).fit(X_train, y_train)
dTreeCla=GridSearchCV(DecisionTreeClassifier(), gridDTreeCla, cv=2).fit(X_train, y_train)
kNC=GridSearchCV(KNeighborsClassifier(), girdKNC, cv=2).fit(X_train, y_train)
lDA=GridSearchCV(LinearDiscriminantAnalysis(),gridLDA, cv=2).fit(X_train, y_train)
qDA=GridSearchCV(QuadraticDiscriminantAnalysis(), gridQDA, cv=2).fit(X_train, y_train)
adaB=GridSearchCV(AdaBoostClassifier(), {'n_estimators': [20]}, cv=2).fit(X_train, y_train)


c:\Users\Usuario\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Usuario\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_i

### SVM

In [8]:
svmResults = pd.DataFrame(svm.cv_results_)
svmResults.sort_values('rank_test_score', ascending=True)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
2,1.423886,0.018266,0.084002,0.000841,2,linear,"{'C': 2, 'kernel': 'linear'}",0.754836,0.750744,0.752790,0.002046,1
1,0.738571,0.022964,0.084229,0.003539,1,linear,"{'C': 1, 'kernel': 'linear'}",0.754836,0.750372,0.752604,0.002232,2
3,2.307863,0.402638,0.075066,0.000167,5,linear,"{'C': 5, 'kernel': 'linear'}",0.754092,0.749628,0.751860,0.002232,3
0,0.315908,0.043546,0.124990,0.008289,0.1,linear,"{'C': 0.1, 'kernel': 'linear'}",0.746280,0.741443,0.743862,0.002418,4


### Random Forest

In [9]:
rfResults = pd.DataFrame(randomForest.cv_results_)
rfResults.sort_values('rank_test_score', ascending=True)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
0,0.038689,0.000698,0.005479,0.000479,10,20,"{'max_depth': 10, 'n_estimators': 20}",0.844494,0.816592,0.830543,0.013951,1


### Logistic Regresion

In [10]:
lgResults=pd.DataFrame(logReg.cv_results_)
lgResults.sort_values('rank_test_score', ascending=True)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,param_solver,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
0,0.033767,0.010813,0.000999,0.000000,5,l2,lbfgs,"{'C': 5, 'penalty': 'l2', 'solver': 'lbfgs'}",0.752604,0.751860,0.752232,0.000372,1
2,0.026006,0.001113,0.001758,0.000758,1.0,l2,lbfgs,"{'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}",0.752232,0.750744,0.751488,0.000744,2
1,0.024031,0.003716,0.000994,0.000003,2,l2,lbfgs,"{'C': 2, 'penalty': 'l2', 'solver': 'lbfgs'}",0.752232,0.749256,0.750744,0.001488,3
3,0.024966,0.000740,0.001706,0.000715,0.1,l2,lbfgs,"{'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}",0.748884,0.743304,0.746094,0.002790,4


### Naive Bayes

In [11]:
gNBResults=pd.DataFrame(gNB.cv_results_)
gNBResults.sort_values('rank_test_score', ascending=True)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_var_smoothing,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
8,0.003017,0.001019,0.003516,1.515746e-03,0.0,{'var_smoothing': 1e-08},0.725446,0.716146,0.720796,0.004650,1
9,0.002001,0.000001,0.002514,5.166531e-04,0.0,{'var_smoothing': 1e-09},0.725074,0.716146,0.720610,0.004464,2
7,0.003093,0.000926,0.002102,1.034737e-04,0.0,{'var_smoothing': 1e-07},0.724330,0.715402,0.719866,0.004464,3
6,0.003887,0.000112,0.004091,1.940489e-03,0.000001,{'var_smoothing': 1e-06},0.718006,0.714658,0.716332,0.001674,4
2,0.001529,0.000528,0.001751,2.484322e-04,0.01,{'var_smoothing': 0.01},0.717262,0.708333,0.712798,0.004464,5
5,0.003001,0.000002,0.002430,4.312992e-04,0.00001,{'var_smoothing': 1e-05},0.710565,0.713542,0.712054,0.001488,6
4,0.004282,0.001280,0.002281,2.849102e-04,0.0001,{'var_smoothing': 0.0001},0.707217,0.708705,0.707961,0.000744,7
1,0.002502,0.000498,0.001039,3.826618e-05,0.1,{'var_smoothing': 0.1},0.696429,0.684524,0.690476,0.005952,8
3,0.001997,0.000003,0.002000,4.768372e-07,0.001,{'var_smoothing': 0.001},0.678199,0.692336,0.685268,0.007068,9
0,0.002313,0.000313,0.000999,5.960464e-07,1.0,{'var_smoothing': 1.0},0.665923,0.666295,0.666109,0.000186,10


### Arboles de decisión

In [12]:
dTreeClaResults=pd.DataFrame(dTreeCla.cv_results_)
dTreeClaResults.sort_values('rank_test_score', ascending=True)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
4,0.006329,0.000329,0.001500,5.002022e-04,gini,8,"{'criterion': 'gini', 'max_depth': 8}",0.827381,0.826637,0.827009,0.000372,1
5,0.006827,0.000172,0.001001,9.536743e-07,gini,9,"{'criterion': 'gini', 'max_depth': 9}",0.832217,0.821801,0.827009,0.005208,1
6,0.006005,0.000007,0.001002,0.000000e+00,gini,10,"{'criterion': 'gini', 'max_depth': 10}",0.824405,0.825893,0.825149,0.000744,3
3,0.005322,0.000323,0.001500,4.997253e-04,gini,7,"{'criterion': 'gini', 'max_depth': 7}",0.807292,0.833705,0.820499,0.013207,4
2,0.004324,0.000323,0.000999,2.384186e-07,gini,6,"{'criterion': 'gini', 'max_depth': 6}",0.808408,0.827381,0.817894,0.009487,5
1,0.004500,0.000500,0.001000,1.072884e-06,gini,5,"{'criterion': 'gini', 'max_depth': 5}",0.797619,0.786830,0.792225,0.005394,6
0,0.004079,0.000080,0.001758,7.585287e-04,gini,4,"{'criterion': 'gini', 'max_depth': 4}",0.767485,0.778646,0.773065,0.005580,7


### Neighbor Classifier

In [13]:
kNCResults=pd.DataFrame(kNC.cv_results_)
kNCResults.sort_values('rank_test_score', ascending=True)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_leaf_size,param_n_neighbors,param_p,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
58,0.006537,0.000461,0.060059,0.004685,6,5,1,"{'leaf_size': 6, 'n_neighbors': 5, 'p': 1}",0.803943,0.800967,0.802455,0.001488,1
78,0.006227,0.000236,0.048748,0.001406,8,5,1,"{'leaf_size': 8, 'n_neighbors': 5, 'p': 1}",0.803943,0.800967,0.802455,0.001488,1
18,0.006912,0.000688,0.055827,0.001855,2,5,1,"{'leaf_size': 2, 'n_neighbors': 5, 'p': 1}",0.803943,0.800967,0.802455,0.001488,1
38,0.008323,0.000348,0.079790,0.009750,4,5,1,"{'leaf_size': 4, 'n_neighbors': 5, 'p': 1}",0.803943,0.800967,0.802455,0.001488,1
68,0.006337,0.000454,0.050175,0.002783,7,5,1,"{'leaf_size': 7, 'n_neighbors': 5, 'p': 1}",0.803943,0.800967,0.802455,0.001488,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,0.006876,0.000877,0.057395,0.000264,7,1,2,"{'leaf_size': 7, 'n_neighbors': 1, 'p': 2}",0.770461,0.760417,0.765439,0.005022,82
1,0.008005,0.000009,0.076295,0.000218,1,1,2,"{'leaf_size': 1, 'n_neighbors': 1, 'p': 2}",0.770461,0.760417,0.765439,0.005022,82
21,0.006500,0.000500,0.053206,0.002166,3,1,2,"{'leaf_size': 3, 'n_neighbors': 1, 'p': 2}",0.770461,0.760417,0.765439,0.005022,82
11,0.007598,0.000403,0.059424,0.002188,2,1,2,"{'leaf_size': 2, 'n_neighbors': 1, 'p': 2}",0.770461,0.760417,0.765439,0.005022,82


### Analisis de discriminación lineal

In [14]:
lDAResults=pd.DataFrame(lDA.cv_results_)
lDAResults.sort_values('rank_test_score', ascending=True)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_solver,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
0,0.007002,0.004003,0.001001,2.384186e-07,svd,{'solver': 'svd'},0.75558,0.748512,0.752046,0.003534,1


### Analisis de discriminación cuadratico

In [15]:
qDAResults=pd.DataFrame(qDA.cv_results_)
qDAResults.sort_values('rank_test_score', ascending=True)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_reg_param,param_store_covariance,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
4,0.002191,1.913309e-04,0.001184,2.099276e-04,0.001,True,"{'reg_param': 0.001, 'store_covariance': True}",0.747396,0.727307,0.737351,0.010045,1
5,0.002011,5.960464e-06,0.000988,7.390976e-06,0.001,False,"{'reg_param': 0.001, 'store_covariance': False}",0.747396,0.727307,0.737351,0.010045,1
2,0.001985,1.430511e-05,0.001537,4.603863e-04,0.0001,True,"{'reg_param': 0.0001, 'store_covariance': True}",0.748884,0.723958,0.736421,0.012463,3
3,0.002008,9.417534e-06,0.000999,2.384186e-07,0.0001,False,"{'reg_param': 0.0001, 'store_covariance': False}",0.748884,0.723958,0.736421,0.012463,3
0,0.001968,3.266335e-05,0.002032,1.032829e-03,0.00001,True,"{'reg_param': 1e-05, 'store_covariance': True}",0.746652,0.725446,0.736049,0.010603,5
1,0.001492,5.081892e-04,0.000509,5.093813e-04,0.00001,False,"{'reg_param': 1e-05, 'store_covariance': False}",0.746652,0.725446,0.736049,0.010603,5
6,0.001001,4.768372e-07,0.001507,4.935265e-04,0.01,True,"{'reg_param': 0.01, 'store_covariance': True}",0.739955,0.732143,0.736049,0.003906,7
7,0.002432,3.954172e-04,0.001602,5.470514e-04,0.01,False,"{'reg_param': 0.01, 'store_covariance': False}",0.739955,0.732143,0.736049,0.003906,7
8,0.002039,6.556511e-06,0.000966,1.788139e-05,0.1,True,"{'reg_param': 0.1, 'store_covariance': True}",0.728423,0.725074,0.726749,0.001674,9
9,0.001251,2.505779e-04,0.002004,2.622604e-06,0.1,False,"{'reg_param': 0.1, 'store_covariance': False}",0.728423,0.725074,0.726749,0.001674,9


### AdaBooster

In [16]:
adaBResults=pd.DataFrame(adaB.cv_results_)
adaBResults.sort_values('rank_test_score', ascending=True)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
0,0.037976,0.000728,0.006875,0.000902,20,{'n_estimators': 20},0.834449,0.827381,0.830915,0.003534,1


### El mejor modelo es el AdaBooster

## Predicción y metricas del modelo

In [17]:
y_preds_adabooster = adaB.predict(X_test)

In [23]:

conf_matrix = pd.crosstab(y_test, y_preds_adabooster, rownames=["observación"], colnames=["Predicción"])
TP = conf_matrix.iloc[1,1]
TN = conf_matrix.iloc[0,0]
FN = conf_matrix.iloc[1,0]
FP = conf_matrix.iloc[0,1]
print("Accuracy: ", accuracy_score(y_test, y_preds_adabooster))
print("Matriz de Confusión: \n\n", conf_matrix)
print("\n\n Sentitividad: ", TP/(TP+FN))
print("Especificidad: ", TN/(TN+FP))
print("\n\n ROC-ACU -> ADABOOSTER = ", roc_auc_score(y_test, y_preds_adabooster))
numpy_array = np.array([[accuracy_score(y_test, y_preds_adabooster), 'Accuracy'], [roc_auc_score(y_test, y_preds_adabooster), 'ROC-ACU'],
                       [TP/(TP+FN), 'Sentitividad'], [TN/(TN+FP), 'Especificidad']])
resultadosMetricas = pd.DataFrame(numpy_array)


Accuracy:  0.84765625
Matriz de Confusión: 

 Predicción      0    1
observación           
0            1384  155
1             196  569


 Sentitividad:  0.7437908496732026
Especificidad:  0.8992852501624431


 ROC-ACU -> ADABOOSTER =  0.8215380499178229


In [28]:
resultadosMetricas.to_csv("Resultados_Metricas.csv")
y_preds_adabooster=pd.DataFrame(y_preds_adabooster)
y_preds_adabooster.to_csv("Predicciones.csv")